In [1]:
import pandas as pd
import requests
import os 
from datetime import datetime, timedelta
import ee 
import pandas as pd
pd.options.mode.chained_assignment
import numpy as np
import geemap 

ee.Initialize()
#ee.Authenticate()

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.jp-Cell { margin-left: -65% !important; margin-right: -65% !important; }</style>"))

In [3]:
directory = 'lakes'
d = {'PointID': ['NW-88021658','NW-88010015','NW-88005133'], 'latitude': [54.426701,54.655347,54.519881], 'longitude': [-3.319169, -3.221301, -3.372862]}
locations = pd.DataFrame(data=d)

In [4]:
locations

,PointID,latitude,longitude
0,NW-88021658,54.426701,-3.319169
1,NW-88010015,54.655347,-3.221301
2,NW-88005133,54.519881,-3.372862


In [5]:
frames = []
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        df = pd.read_csv(f, parse_dates = True)
        df = df.loc[(df['determinand.notation'] == 301) & ((df['sample.samplingPoint.notation'] == 'NW-88021658') | (df['sample.samplingPoint.notation'] == 'NW-88010015')  | (df['sample.samplingPoint.notation'] == 'NW-88005133' )) ]        
        df = df[['sample.samplingPoint.notation','determinand.label','result','determinand.unit.label','sample.sampleDateTime']]
        df.rename(columns={'sample.samplingPoint.notation': 'PointID', 'determinand.label': 'Property', 'determinand.unit.label': 'Unit','sample.sampleDateTime':'SampleDate', 'result': 'Result'}, inplace=True)
        frames.append(df)

In [6]:
result = pd.concat(frames)

In [7]:
result = pd.merge(result, locations.set_index('PointID'), left_on= 'PointID',
                   right_index= True, 
                   how = 'left')

In [8]:
df = result

In [9]:
df

,PointID,Property,Result,Unit,SampleDate,latitude,longitude
30874,NW-88005133,C - Org Filt,1.28,mg/l,2018-01-22T11:49:00,54.519881,-3.372862
30914,NW-88005133,C - Org Filt,1.33,mg/l,2018-02-22T10:57:00,54.519881,-3.372862
30933,NW-88005133,C - Org Filt,0.84,mg/l,2018-04-03T13:16:00,54.519881,-3.372862
30969,NW-88005133,C - Org Filt,0.97,mg/l,2018-04-27T11:17:00,54.519881,-3.372862
31027,NW-88005133,C - Org Filt,1.41,mg/l,2018-05-25T10:55:00,54.519881,-3.372862
...,...,...,...,...,...,...,...
94622,NW-88021658,C - Org Filt,0.54,mg/l,2023-11-10T11:55:00,54.426701,-3.319169
11852,NW-88005133,C - Org Filt,0.98,mg/l,2024-01-24T13:59:00,54.519881,-3.372862
21535,NW-88010015,C - Org Filt,1.70,mg/l,2024-01-26T10:15:00,54.655347,-3.221301
21741,NW-88010015,C - Org Filt,2.10,mg/l,2024-03-26T10:28:00,54.655347,-3.221301


In [9]:
df['SampleDate'] = df['SampleDate'].apply(lambda x: x.split('T', 1)[0])

In [10]:
df['SampleDate'] = df['SampleDate'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').strftime('%Y-%m-%d'))

In [11]:
df['SampleDate'].unique()

array(['2018-01-22', '2018-02-22', '2018-04-03', '2018-04-27',
       '2018-05-25', '2018-06-28', '2018-07-27', '2018-08-23',
       '2018-09-26', '2018-10-25', '2018-11-26', '2018-12-10',
       '2018-01-19', '2018-02-21', '2018-03-21', '2018-04-26',
       '2018-05-23', '2018-06-26', '2018-07-18', '2018-08-15',
       '2018-09-19', '2018-10-22', '2018-11-20', '2018-12-04',
       '2018-01-11', '2018-02-20', '2018-03-20', '2018-04-18',
       '2018-05-21', '2018-07-10', '2018-07-19', '2018-08-24',
       '2018-09-27', '2018-10-29', '2018-11-15', '2018-12-28',
       '2019-01-23', '2019-02-27', '2019-03-20', '2019-04-24',
       '2019-05-28', '2019-06-18', '2019-07-26', '2019-08-28',
       '2019-09-30', '2019-10-23', '2019-11-25', '2019-12-17',
       '2019-01-08', '2019-02-21', '2019-03-21', '2019-04-11',
       '2019-05-15', '2019-06-14', '2019-07-17', '2019-08-16',
       '2019-09-25', '2019-10-29', '2019-11-18', '2019-12-11',
       '2019-03-29', '2019-05-24', '2019-06-13', '2019-

In [12]:
df = df.sort_values(by=['SampleDate'])

In [13]:
df1 = df
df1.insert(0,'MY_ID',range(0,len(df1)))
df1

,MY_ID,PointID,Property,Result,Unit,SampleDate,latitude,longitude
82047,0,NW-88021658,C - Org Filt,0.58,mg/l,2018-01-11,54.426701,-3.319169
64417,1,NW-88010015,C - Org Filt,1.61,mg/l,2018-01-19,54.655347,-3.221301
30874,2,NW-88005133,C - Org Filt,1.28,mg/l,2018-01-22,54.519881,-3.372862
82066,3,NW-88021658,C - Org Filt,0.67,mg/l,2018-02-20,54.426701,-3.319169
64438,4,NW-88010015,C - Org Filt,1.67,mg/l,2018-02-21,54.655347,-3.221301
...,...,...,...,...,...,...,...,...
40321,162,NW-88005133,C - Org Filt,1.00,mg/l,2023-12-11,54.519881,-3.372862
28388,163,NW-88021658,C - Org Filt,0.61,mg/l,2024-01-08,54.426701,-3.319169
11852,164,NW-88005133,C - Org Filt,0.98,mg/l,2024-01-24,54.519881,-3.372862
21535,165,NW-88010015,C - Org Filt,1.70,mg/l,2024-01-26,54.655347,-3.221301


In [14]:
feature = ee.FeatureCollection("projects/ee-konradbujak09/assets/lakes_analysis", {})
roi = feature.geometry()

In [15]:
def mask_clouds(img):
  qa_band = 'cs_cdf'
  clear_treshold = 0.65
  return img.updateMask(img.select(qa_band).gte(clear_treshold))

In [16]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('ImgDate').toInt())

In [17]:
def getDOC(image):
    doc = image.expression(
        '432 * exp(-2.24*GREEN/RED)', {
        'GREEN': image.select('B3'),
        'RED': image.select('B4')}).rename('DOC')
    image = image.addBands(doc)

    return(image)

In [18]:
def getCDOM(image):
    cdom = image.expression(
        '537 * exp(-2.93*GREEN/RED)', {
        'GREEN': image.select('B3'),
        'RED': image.select('B4')}).rename('CDOM')
    image = image.addBands(cdom)

    return(image)

In [19]:
def getTURB(image):
    turbidity = image.expression(
        '8.93 * (GREEN/AERO) - 6.39', {
        'AERO': image.select('B1'),
        'GREEN': image.select('B3')}).rename('TURB')
    image = image.addBands(turbidity)

    return(image)

In [20]:
def getCHLA(image):
    chla = image.expression(
        '4.26 * pow((GREEN/AERO), 3.94)', {
        'AERO': image.select('B1'),
        'GREEN': image.select('B3')}).rename('CHLA')
    image = image.addBands(chla)

    return(image)

In [21]:
def getNDVI(image):
    ndvi = image.normalizedDifference(["B8", "B4"]).rename('NDVI')
    image = image.addBands(ndvi)

    return(image)

In [22]:
df1

,MY_ID,PointID,Property,Result,Unit,SampleDate,latitude,longitude
82047,0,NW-88021658,C - Org Filt,0.58,mg/l,2018-01-11,54.426701,-3.319169
64417,1,NW-88010015,C - Org Filt,1.61,mg/l,2018-01-19,54.655347,-3.221301
30874,2,NW-88005133,C - Org Filt,1.28,mg/l,2018-01-22,54.519881,-3.372862
82066,3,NW-88021658,C - Org Filt,0.67,mg/l,2018-02-20,54.426701,-3.319169
64438,4,NW-88010015,C - Org Filt,1.67,mg/l,2018-02-21,54.655347,-3.221301
...,...,...,...,...,...,...,...,...
40321,162,NW-88005133,C - Org Filt,1.00,mg/l,2023-12-11,54.519881,-3.372862
28388,163,NW-88021658,C - Org Filt,0.61,mg/l,2024-01-08,54.426701,-3.319169
11852,164,NW-88005133,C - Org Filt,0.98,mg/l,2024-01-24,54.519881,-3.372862
21535,165,NW-88010015,C - Org Filt,1.70,mg/l,2024-01-26,54.655347,-3.221301


In [23]:
DOC = [] #l1C
TURB = [] #L2A
NDVI = [] #L2A
CHLA = [] #L2A
CDOM = [] #L1C
ImageDate = []
Indexes = []

for index,row in df1.iterrows():
    features = []
    start_date = datetime.strptime(row['SampleDate'], '%Y-%m-%d') - timedelta(days=4)
    end_date = datetime.strptime(row['SampleDate'], '%Y-%m-%d') + timedelta(days=4)
    start_date = start_date.strftime('%Y-%m-%d')
    end_date = end_date.strftime('%Y-%m-%d')
    
    csPlus = ee.ImageCollection('GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED')

    qa_band = 'cs_cdf'

    img = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
    .filterBounds(roi) \
    .filterDate(start_date, end_date) \
    .map(addDate).linkCollection(csPlus, [qa_band]).map(mask_clouds).map(getDOC).map(getCDOM)\
    .sort('CLOUDY_PIXEL_PERCENTAGE').first()

    img_sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(roi) \
    .filterDate(start_date, end_date) \
    .map(addDate).linkCollection(csPlus, [qa_band]).map(mask_clouds).map(getNDVI).map(getTURB).map(getCHLA)\
    .sort('CLOUDY_PIXEL_PERCENTAGE').first()
 

    point = ee.Geometry.Point([row['longitude'], row['latitude']])
    buffer = point.buffer(distance = 10)
    properties = dict(row)
    feature = ee.Feature(buffer, properties)
    features.append(feature)
    collection = ee.FeatureCollection(features)

    stats = img.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=collection,
    scale=10,  # meters
        )

    stats_sr = img_sr.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=collection,
    scale=10,  # meters
        )

    DOC.append(stats.get('DOC').getInfo())
    TURB.append(stats_sr.get('TURB').getInfo())
    NDVI.append(stats_sr.get('NDVI').getInfo())
    CDOM.append(stats.get('CDOM').getInfo())
    CHLA.append(stats_sr.get('CHLA').getInfo())
    ImageDate.append(str(stats.get('ImgDate').getInfo()))
    print(stats.get('ImgDate').getInfo())
    Indexes.append(row['MY_ID'])
    



2018.0108
None
None
None
None
2018.0222
2018.0319
2018.0319
None
2018.042
2018.0428
None
2018.0523
2018.0523
2018.0528
2018.0629
2018.0629
None
2018.0719
2018.0719
None
None
None
None
None
None
2018.0927
None
2018.1027
2018.1027
None
None
2018.1126
None
2018.1209
2018.1224
2019.0108
2019.0123
2019.0224
2019.0227
2019.0227
None
None
2019.0331
2019.041
2019.042
2019.0515
None
2019.0528
None
None
None
None
None
None
None
2019.0813
None
2019.0907
2019.092
None
2019.1002
2019.1017
None
2019.103
2019.1114
None
None
2019.1209
2019.1209
None
2020.0103
None
None
None
2020.0227
2020.0227
2020.0305
None
2020.032
2021.0424
2021.0424
2021.0424
None
2021.0519
None
2021.0618
None
None
2021.0716
2021.0716
2021.0721
None
None
2021.0929
None
None
2021.1021
None
None
None
2021.1205
None
None
None
None
None
None
None
None
2022.032
2022.0325
2022.0325
2022.0424
2022.0424
2022.0429
None
2022.0519
2022.0519
2022.0623
None
None
2022.0718
None
None
2022.0812
2022.0812
2022.0812
None
None
2022.0924
None
None
No

In [26]:
d = {'MY_ID': Indexes, 'DOC': DOC, 'Turbidity': TURB, 'NDVI': NDVI, 'Chlorophyll': CHLA, 'CDOM': CDOM, 'ImageDate': ImageDate}
values = pd.DataFrame(d)

In [27]:
values

,MY_ID,DOC,Turbidity,NDVI,Chlorophyll,CDOM,ImageDate
0,0,0.001221,-0.000164,-0.000017,0.000044,0.000507,2018.0108
1,1,NaN,NaN,NaN,NaN,NaN,None
2,2,NaN,NaN,NaN,NaN,NaN,None
3,3,NaN,NaN,NaN,NaN,NaN,None
4,4,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...
162,162,NaN,NaN,NaN,NaN,NaN,None
163,163,0.001071,-0.000214,0.000100,0.000044,0.000427,2024.0107
164,164,NaN,NaN,NaN,NaN,NaN,None
165,165,NaN,NaN,NaN,NaN,NaN,None


In [35]:
result = pd.merge(df1, values.set_index('MY_ID'), left_on= 'MY_ID',
                   right_index= True, 
                   how = 'left')

In [36]:
result = result.dropna()

In [37]:
result['ImageDate'] = result['ImageDate'].apply(lambda x: str(x).replace('.', ''))

In [38]:
result['ImageDate'] = result['ImageDate'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
result['ImageDate'] = result['ImageDate'].apply(lambda x: datetime.strftime(x,'%Y-%m-%d'))

In [39]:
result = result.drop('MY_ID', axis=1)

In [40]:
title = datetime.today().strftime('%Y-%m-%d')
result.to_csv(title+'.csv', index=False)  